In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("topicmodelling.csv",encoding='latin1')
df

,date,username,created_at,tweet,heartbreak,Friendship issue,Work issues,Suicidal thoughts,insomania,looks_insecurity,mental health issue,family issue
0,18-03-2021,amanda_mental,2021-03-18 21:59:02 India Standard Time,i am worthless im a horrible person,0.0,0.055779,0.0,0.054988,0.0,0.0,0.058299,0.0
1,18-03-2021,amanda_mental,2021-03-18 08:44:03 India Standard Time,im open to new stuff so whats your fav,0.0,0.059721,0.0,0.029437,0.0,0.0,0.000000,0.0
2,18-03-2021,amanda_mental,2021-03-18 08:35:19 India Standard Time,dyed my hair pink,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
3,18-03-2021,amanda_mental,2021-03-18 08:34:57 India Standard Time,i want new music any suggestions,0.0,0.000000,0.0,0.034339,0.0,0.0,0.000000,0.0
4,18-03-2021,amanda_mental,2021-03-18 08:25:42 India Standard Time,me in target earlier,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6563,19-11-2020,thesanjunipero,2020-11-19 13:31:22 India Standard Time,o okay i uh mm,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
6564,18-11-2020,thesanjunipero,2020-11-18 18:07:53 India Standard Time,dicari asisten pribadi jobdesk ngerebus bawang...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
6565,17-11-2020,thesanjunipero,2020-11-17 18:04:06 India Standard Time,win win solution,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
6566,17-11-2020,thesanjunipero,2020-11-17 10:38:21 India Standard Time,bikin showreel aesthetic but ngerebus kulit ja...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0


In [3]:
df_modified=df.drop(['date','created_at','tweet','username'],axis=1)

In [4]:
df_modified.head()

,heartbreak,Friendship issue,Work issues,Suicidal thoughts,insomania,looks_insecurity,mental health issue,family issue
0,0.0,0.055779,0.0,0.054988,0.0,0.0,0.058299,0.0
1,0.0,0.059721,0.0,0.029437,0.0,0.0,0.000000,0.0
2,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
3,0.0,0.000000,0.0,0.034339,0.0,0.0,0.000000,0.0
4,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0


In [5]:
df_modified['unidentified'] = df_modified.any(axis=1)
df_modified['unidentified'] = df_modified["unidentified"].astype(int)
for i in range(len(df_modified)):
    if df_modified['unidentified'][i] == 1:
        df_modified['unidentified'][i] = 0.0
    else:
        df_modified['unidentified'][i] = 1.0

<ipython-input-5-649e83730b6a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modified['unidentified'][i] = 0.0
<ipython-input-5-649e83730b6a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modified['unidentified'][i] = 1.0


In [6]:
df_modified['dominant_value']=df_modified.max(axis=1)

In [7]:
df_modified.head()

,heartbreak,Friendship issue,Work issues,Suicidal thoughts,insomania,looks_insecurity,mental health issue,family issue,unidentified,dominant_value
0,0.0,0.055779,0.0,0.054988,0.0,0.0,0.058299,0.0,0,0.058299
1,0.0,0.059721,0.0,0.029437,0.0,0.0,0.000000,0.0,0,0.059721
2,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1,1.000000
3,0.0,0.000000,0.0,0.034339,0.0,0.0,0.000000,0.0,0,0.034339
4,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1,1.000000


In [8]:
(df_modified['dominant_value'] == 0).sum()

0

In [9]:
 df_modified['dominant_value']= df_modified['dominant_value'].replace(0, np.nan)

In [10]:
df_modified.isnull().sum()

heartbreak             0
Friendship issue       0
Work issues            0
Suicidal thoughts      0
insomania              0
looks_insecurity       0
mental health issue    0
family issue           0
unidentified           0
dominant_value         0
dtype: int64

In [11]:
df_modified=df_modified.dropna()

In [12]:
df_modified=df_modified.drop(['dominant_value'],axis=1)

In [13]:
df_modified.shape

(6568, 9)

In [14]:
df_modified.head()

,heartbreak,Friendship issue,Work issues,Suicidal thoughts,insomania,looks_insecurity,mental health issue,family issue,unidentified
0,0.0,0.055779,0.0,0.054988,0.0,0.0,0.058299,0.0,0
1,0.0,0.059721,0.0,0.029437,0.0,0.0,0.000000,0.0,0
2,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1
3,0.0,0.000000,0.0,0.034339,0.0,0.0,0.000000,0.0,0
4,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1


In [15]:
#df_modified=df_modified.drop(['date','created_at','tweet','username'],axis=1)

In [16]:
df_modified.shape

(6568, 9)

In [17]:
df_modified['domiant_topic']=df_modified.idxmax(axis=1)

In [18]:
df_modified

,heartbreak,Friendship issue,Work issues,Suicidal thoughts,insomania,looks_insecurity,mental health issue,family issue,unidentified,domiant_topic
0,0.0,0.055779,0.0,0.054988,0.0,0.0,0.058299,0.0,0,mental health issue
1,0.0,0.059721,0.0,0.029437,0.0,0.0,0.000000,0.0,0,Friendship issue
2,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1,unidentified
3,0.0,0.000000,0.0,0.034339,0.0,0.0,0.000000,0.0,0,Suicidal thoughts
4,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1,unidentified
...,...,...,...,...,...,...,...,...,...,...
6563,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1,unidentified
6564,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1,unidentified
6565,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1,unidentified
6566,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1,unidentified


In [19]:
df_modified.domiant_topic.value_counts()

unidentified           1864
Work issues             830
Friendship issue        698
heartbreak              693
Suicidal thoughts       691
mental health issue     647
family issue            466
insomania               399
looks_insecurity        280
Name: domiant_topic, dtype: int64

In [20]:
frequency_table = pd.value_counts(df_modified.domiant_topic).to_frame().reset_index()
frequency_table

,index,domiant_topic
0,unidentified,1864
1,Work issues,830
2,Friendship issue,698
3,heartbreak,693
4,Suicidal thoughts,691
5,mental health issue,647
6,family issue,466
7,insomania,399
8,looks_insecurity,280


In [21]:
frequency_table['Cumulative Frequency'] = frequency_table['domiant_topic'].cumsum()

In [22]:
frequency_table

,index,domiant_topic,Cumulative Frequency
0,unidentified,1864,1864
1,Work issues,830,2694
2,Friendship issue,698,3392
3,heartbreak,693,4085
4,Suicidal thoughts,691,4776
5,mental health issue,647,5423
6,family issue,466,5889
7,insomania,399,6288
8,looks_insecurity,280,6568


In [23]:
frequency_table['relative_frequency']=frequency_table['domiant_topic']/frequency_table['domiant_topic'].sum()

In [24]:
frequency_table

,index,domiant_topic,Cumulative Frequency,relative_frequency
0,unidentified,1864,1864,0.283800
1,Work issues,830,2694,0.126370
2,Friendship issue,698,3392,0.106273
3,heartbreak,693,4085,0.105512
4,Suicidal thoughts,691,4776,0.105207
5,mental health issue,647,5423,0.098508
6,family issue,466,5889,0.070950
7,insomania,399,6288,0.060749
8,looks_insecurity,280,6568,0.042631


In [25]:
frequency_table['cumulativerequency_percentage']=frequency_table['relative_frequency'].cumsum()

In [26]:
frequency_table

,index,domiant_topic,Cumulative Frequency,relative_frequency,cumulativerequency_percentage
0,unidentified,1864,1864,0.283800,0.283800
1,Work issues,830,2694,0.126370,0.410171
2,Friendship issue,698,3392,0.106273,0.516443
3,heartbreak,693,4085,0.105512,0.621955
4,Suicidal thoughts,691,4776,0.105207,0.727162
5,mental health issue,647,5423,0.098508,0.825670
6,family issue,466,5889,0.070950,0.896620
7,insomania,399,6288,0.060749,0.957369
8,looks_insecurity,280,6568,0.042631,1.000000
